In [1]:
## TOPIC Modelling

In [2]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Import Dataset
df= pd.read_csv('/Users/vibhuverma/Desktop/CLASSWORK/BUSINESS PRACTICUM/Anti-Corruption/Data Provided/USAID_Project_Data/USAID_Anticorruption_Projects_Database.csv')
df = df.drop(['rfp_rftop','final_res','mind_eval','final_eval','audit','interim_rep','addl_docs','addl_info'],axis=1)

In [4]:
# Convert to list
data = df.proj_desc.values.tolist()

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['The five components of this program seek to take what already exists and '
 'improve upon it, implement it and find ways to ensure that the progress '
 'achieved is sustainable over the long term. The project will help the '
 'government of the Islamic Republic of Afghanistan strengthen its High Office '
 'of Oversight (HOO) - making it a strong, effective institution that is able '
 'to lead, monitor, coordinate and report on efforts to combat corruption '
 'across the country. The project will work and implement a program of '
 'institutional development and sustainability. It will also support the '
 'office in carrying out its priority responsibilities in asset registration '
 'and verification, complaints management and case tracking, and coordination '
 'and monitoring of anticorruption performance across other government '
 'agencies.']


In [5]:
#tokenize and clean up using gensim 

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))


In [6]:
#lemmatization

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['component program seek take already exist improve implement find way ensure progress achieve sustainable long term project will help government strengthen high office oversight hoo make strong effective institution able lead monitor coordinate report effort combat corruption country project will work implement program institutional development sustainability will also support office carry priority asset registration verification complaint management case tracking coordination monitoring anticorruption performance other government agency', 'overall goal program support rule law strategy specifically pillar improve justice sector reduce corruption pillar build leadership justice sector civil society support pillar contractor attempt develop capacity judiciary law school pillar contractor work raise public awareness encourage citizen resolve dispute formal justice sector program divide component capacity building judiciary capacity building court management system capacity building facu

In [7]:
## TFIDF


vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             ngram_range=(1,3)             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [8]:
#Checking the sparcity

# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  9.346462864981383 %


In [9]:
## BASE MODEL


# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=42,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=42)


In [10]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -11400.430736159025
Perplexity:  1799.893102521426
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 42,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [11]:
## HYPERPARAMETER TUNING

# Define Search Param
search_params = {'n_components': np.arange(5,30).tolist(), 'learning_decay': [.3,.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.3, 0.5, 0.7, 0.9],
                         'n_components': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                                          16, 17, 18, 19, 20, 21, 22, 23, 24,
                                          25, 26, 27, 28, 29]})

In [12]:
## BEST TOPIC MODEL


# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))


Best Model's Params:  {'learning_decay': 0.7, 'n_components': 5}
Best Log Likelihood Score:  -2482.404402934444
Model Perplexity:  837.4458886529025


In [13]:
### DOMINANT TOPIC IN EACH DOCUMENT


# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics




,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.890000,0.030000,0.030000,0.030000,0.030000,0
Doc1,0.870000,0.030000,0.030000,0.030000,0.030000,0
Doc2,0.810000,0.050000,0.050000,0.050000,0.050000,0
Doc3,0.300000,0.390000,0.210000,0.050000,0.040000,1
Doc4,0.830000,0.040000,0.060000,0.040000,0.040000,0
Doc5,0.030000,0.030000,0.870000,0.030000,0.030000,2
Doc6,0.430000,0.330000,0.180000,0.030000,0.030000,0
Doc7,0.570000,0.040000,0.310000,0.040000,0.040000,0
Doc8,0.040000,0.040000,0.850000,0.040000,0.040000,2
Doc9,0.030000,0.030000,0.880000,0.030000,0.030000,2


In [14]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      150.222153  -72.269554       1        1  44.261168
1       51.163364 -159.968018       2        1  32.030424
2      -73.144554   21.523815       3        1  16.097373
3      -66.410133 -110.606438       4        1   3.815545
4       54.417324   12.443444       5        1   3.795490, topic_info=                            Term       Freq      Total Category  logprob  \
134                       health   5.000000   5.000000  Default  30.0000   
180                        local  15.000000  15.000000  Default  29.0000   
231                  procurement   4.000000   4.000000  Default  28.0000   
182             local government   7.000000   7.000000  Default  27.0000   
195                 municipality   6.000000   6.000000  Default  26.0000   
..                           ...        ...        ...      ...      ...   
317  transparency accountability   0.160163   4.545128   Topic5  -5.8873   
4                        achieve   0.159674   3.870446   Topic5  -5.8904   
245                 project work   0.159630   4.148262   Topic5  -5.8906   
316                 transparency   0.159580   9.380537   Topic5  -5.8910   
244              project support   0.159259   3.615898   Topic5  -5.8930   

     loglift  
134  30.0000  
180  29.0000  
231  28.0000  
182  27.0000  
195  26.0000  
..       ...  
317  -0.0743  
4     0.0834  
245   0.0138  
316  -0.8025  
244   0.1488  

[282 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1         1  0.890071       access
4         1  0.258368      achieve
4         2  0.516736      achieve
4         3  0.258368      achieve
7         1  0.306454        actor
...     ...       ...          ...
318       3  0.370246  transparent
319       2  0.414168         unit
319       3  0.414168         unit
321       2  0.466935        woman
321       3  0.466935        woman

[345 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5])

In [15]:
### TOPIC KEYWORDS ###

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,ability,access,accountability,accountable,achieve,action,activity,actor,address,administration,...,training,transition,transparency,transparency accountability,transparent,unit,use,woman,work,year
Topic0,1.900965,5.462092,5.127482,2.474513,1.661958,0.662229,4.516686,1.011864,3.459049,3.278972,...,4.342576,2.874204,6.206919,2.418130,3.004516,0.385585,2.942717,0.214244,5.516964,1.626097
Topic1,0.637935,0.332635,2.082393,0.201393,1.875577,1.891366,1.757804,2.356644,1.790609,2.055778,...,3.866680,0.201762,2.854585,1.495795,0.842585,1.025492,1.563304,2.629736,2.518316,1.061516
Topic2,0.535617,0.573654,1.706735,2.009596,0.761263,0.200800,0.881903,0.200867,0.345742,0.498446,...,0.275064,0.202565,1.876841,1.194370,2.297621,1.137074,0.396213,1.980459,1.557996,0.275946
Topic3,0.200344,0.200977,0.200470,0.200111,0.200741,0.201445,0.200277,0.202654,0.201438,0.200164,...,0.200177,0.200157,0.200367,0.200805,0.200606,0.201180,0.201717,0.203466,0.200747,0.200189
Topic4,0.200186,0.200944,0.201766,0.200108,0.202578,0.200559,0.200343,0.200155,0.200197,0.201241,...,0.200173,0.200092,0.202460,0.203199,0.201926,0.200272,0.201917,0.200240,0.200467,0.200185


In [16]:
### TOP 15 KEYWORDS FOR EACH TOPIC ###

# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,strengthen,program,project,corruption,justice,society,judicial,capacity,civil society,civil,increase,public,law,legal,community
Topic 1,corruption,project,support,reform,assistance,improve,business,technical,government,technical assistance,trade,economic,provide,program,investment
Topic 2,local,local government,municipality,municipal,government,service,governance,local governance,citizen,budget,level,capacity,delivery,participation,local level
Topic 3,health,afghan,long,democracy,project help,participation,structure,international,prevent,local level,respond,lead,public private,basic,financial
Topic 4,public procurement,procurement,project design,design,right,identify,improve public,project,approach,control,advocate,public,authority,internal,health


In [17]:
### Document Clustering with topic model as features ###
tt_matrix = best_lda_model.components_

In [18]:
tt_matrix

array([[1.90096474, 5.4620922 , 5.12748183, ..., 0.21424446, 5.51696442,
        1.62609661],
       [0.63793531, 0.33263547, 2.08239338, ..., 2.62973641, 2.51831603,
        1.06151591],
       [0.53561746, 0.57365377, 1.70673493, ..., 1.98045858, 1.55799594,
        0.2759459 ],
       [0.20034438, 0.20097743, 0.20046956, ..., 0.20346591, 0.20074705,
        0.20018904],
       [0.20018575, 0.2009444 , 0.20176598, ..., 0.20024022, 0.20046679,
        0.20018491]])